In [2]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar"}}

In [3]:
import simple.HTMindex
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1570231724611_0012,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


In [4]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils


read object table parquet files into dataframe (objDF)

create new dataframe (newDF) with calculated htmid

In [5]:
val datafile = "/user/hive/warehouse/object"
val objDF = spark.read.parquet(datafile)

val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl")))
//newDF.printSchema()
//newDF.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/object
objDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 234 more fields]
newDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 235 more fields]


write newDF to external table in SqlDataPool using Spark-SQL connector


In [7]:

println("Use MSSQL connector to write to master SQL instance ")

val servername = "jdbc:sqlserver://master-0.master-svc"
val dbname = "LSST"
var url = servername + ";" + "databaseName=" + dbname + ";"

val user = "sa"
val password = "REDACTED" //Please specify password here

val datapool_table = "ObjectDP_htmid"

val datasource_name = "SqlDataPool"

val batchsize = 1000000

val start = Calendar.getInstance().getTime()

try {
  newDF.write 
    .format("com.microsoft.sqlserver.jdbc.spark") 
    .mode("append") 
    .option("url", url) 
    .option("dbtable", datapool_table) 
    .option("user", user) 
    .option("password", password) 
    .option("dataPoolDataSource",datasource_name)
    .option("batchsize",batchsize)
    .save()
} catch {
    case e: Throwable => println("MSSQL Connector write failed: " + e)
}
val end = Calendar.getInstance().getTime()
println(start)
println(end)
//var duration = end - start
//println("duration:" + duration)
print("MSSQL Connector write(append) to data pool external table succeeded")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…